# Задание 9.5 модуля ML-6

>Обучите модель линейной регрессии на найденных двумя способами трёх важных признаках и сравните полученные результаты. Загрузите полученный ноутбук (в формате IPYNB) в форму ниже.

In [36]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest, f_regression, f_classif, mutual_info_classif, mutual_info_regression, chi2

In [37]:
# загрузим данные
%%capture
!wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx

UsageError: Line magic function `%%capture` not found.


In [38]:
# прочтём данные
data = pd.read_excel('data_ford_price.xlsx') 
# удалим пропуски
data.dropna(inplace = True)
# определим object'овые признаки
columns_to_drop = data.columns[data.dtypes == 'object']
# дропнем эти признаки
data = data.drop(columns_to_drop,axis=1)
# разобъем сет на тренировочный и тестовый
y = data['price']
x = data.drop(columns='price')
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [39]:
# отберём признаки методом RFE
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
 
rfe_selection = selector.get_feature_names_out()
rfe_selection

array(['year', 'condition', 'cylinders'], dtype=object)

In [40]:
# отберём признаки методом SelectKBest
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
 
kbest_selection = selector.get_feature_names_out()
kbest_selection

array(['year', 'condition', 'odometer'], dtype=object)

In [41]:
# обучим регрессию на отобранных признаках
model_rfe = LinearRegression()
model_kbest = LinearRegression()
model_rfe.fit(X_train[rfe_selection], y_train)
model_kbest.fit(X_train[kbest_selection], y_train)
y_predicted_rfe = model_rfe.predict(X_test[rfe_selection])
y_predicted_kbest = model_kbest.predict(X_test[kbest_selection])
mae_rfe = mean_absolute_error(y_test, y_predicted_rfe)
mae_kbest = mean_absolute_error(y_test, y_predicted_kbest)
print('MAE_rfe: %.3f' % mae_rfe)
print('MAE_kbest: %.3f' % mae_kbest)

MAE_rfe: 5103.938
MAE_kbest: 4682.026


## На основе полученных метрик можно сделать вывод, что для тестовой выборки лучший показатель даёт метод отбора `SelectKBest`